# Test Great Expectations con datos de subvenciones
En este cuaderno vamos a modelar las expectativas respecto a los datos que pueden descargarse desde el portal de subvenciones.

Para ello seguiremos los cuatro pasos detallados en el documento: https://docs.greatexpectations.io/docs/tutorials/getting_started/tutorial_setup


## 1.- Instalación

Instalamos GE con `pip install great_expectations`. 

Descargamos los datos desde el canal de Discord. Estos datos se ofrecen en formato csv.gz.

Para usar GE necesitamos un conjunto de datos de entrenamiento y uno de verificación. Como solo tenemos un fichero, vamos a partirlo en dos mitades. Pero antes de nada, vamos a inspeccionarlo manualmente.

In [4]:
import csv
import gzip
import pandas as pd

df = pd.read_csv('convocatorias2.csv.gz', compression='gzip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,531256,329708,NO,CABILDO INSULAR DE TENERIFE,CABILDO INSULAR DE TENERIFE,None,19/01/2017,Subvención con destino a la ejecución de “Inve...,•\thttp://www.bopsantacruzdetenerife.org/2017/...,None,None,352606,1
1,757907,556347,NO,ONDA,AYUNTAMIENTO DE ONDA,None,08/04/2021,Bases reguladoras de la convocatoria de subven...,“www.onda.es/ond/web_php/index.php?contenido=s...,None,None,352606,2
2,537485,335937,NO,DIPUTACIÓN PROV. DE A CORUÑA,"DIPUTACIÓN PROVINCIAL DE CORUÑA, A",None,16/03/2017,Convocatoria del programa de subvenciones diri...,“http://bop.dicoruna.es/bopportal/publicado/20...,None,None,352606,3
3,535468,333920,NO,DIPUTACIÓN PROV. DE A CORUÑA,"DIPUTACIÓN PROVINCIAL DE CORUÑA, A",None,01/03/2017,Programa de subvenciones para limpieza de play...,“http://bop.dicoruna.es/bopportal/publicado/20...,None,None,352606,4
4,638102,436552,NO,DIPUTACIÓN PROV. DE A CORUÑA,"DIPUTACIÓN PROVINCIAL DE CORUÑA, A",None,24/01/2019,Programa de subvenciones para limpieza de play...,“http://bop.dacoruna.gal/bopportal/publicado/2...,None,None,352606,5


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352606 entries, 0 to 352605
Data columns (total 13 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       352606 non-null  int64 
 1   1       352606 non-null  object
 2   2       352606 non-null  object
 3   3       352606 non-null  object
 4   4       352606 non-null  object
 5   5       352606 non-null  object
 6   6       352606 non-null  object
 7   7       352606 non-null  object
 8   8       352606 non-null  object
 9   9       352606 non-null  object
 10  10      352606 non-null  object
 11  11      352606 non-null  int64 
 12  12      352606 non-null  int64 
dtypes: int64(3), object(10)
memory usage: 35.0+ MB


En total tenemos 352606 entradas, que son aproximadamente las mismas que en el dataset original. Como principal diferencia, encontramos en la columna 8 que hay varias entradas que comienzan por comillas o tabuladores. Vamos a ver si podemos limpiarlo, pasando como parámetro quotechar='"' en la importación de los datos.

In [14]:
df2 = pd.read_csv('convocatorias2.csv.gz', compression='gzip', index_col=0, header=None, quotechar='"')
df2.head(20)

,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,,,,,,,,,
531256,329708,NO,CABILDO INSULAR DE TENERIFE,CABILDO INSULAR DE TENERIFE,None,19/01/2017,Subvención con destino a la ejecución de “Inve...,•\thttp://www.bopsantacruzdetenerife.org/2017/...,None,None,352606,1
757907,556347,NO,ONDA,AYUNTAMIENTO DE ONDA,None,08/04/2021,Bases reguladoras de la convocatoria de subven...,“www.onda.es/ond/web_php/index.php?contenido=s...,None,None,352606,2
537485,335937,NO,DIPUTACIÓN PROV. DE A CORUÑA,"DIPUTACIÓN PROVINCIAL DE CORUÑA, A",None,16/03/2017,Convocatoria del programa de subvenciones diri...,“http://bop.dicoruna.es/bopportal/publicado/20...,None,None,352606,3
535468,333920,NO,DIPUTACIÓN PROV. DE A CORUÑA,"DIPUTACIÓN PROVINCIAL DE CORUÑA, A",None,01/03/2017,Programa de subvenciones para limpieza de play...,“http://bop.dicoruna.es/bopportal/publicado/20...,None,None,352606,4
638102,436552,NO,DIPUTACIÓN PROV. DE A CORUÑA,"DIPUTACIÓN PROVINCIAL DE CORUÑA, A",None,24/01/2019,Programa de subvenciones para limpieza de play...,“http://bop.dacoruna.gal/bopportal/publicado/2...,None,None,352606,5
666339,464779,NO,CANARIAS,"CONSEJERÍA DE ECONOMÍA, CONOCIMIENTO Y EMPLEO",None,02/07/2019,“RED INSULAR DE PUNTO DE ATENCIÓN AL EMPRENDED...,“RED INSULAR DE PUNTO DE ATENCIÓN AL EMPRENDED...,None,None,352606,6
784793,583233,NO,CANARIAS,"CONSEJERÍA DE ECONOMÍA, CONOCIMIENTO Y EMPLEO",None,10/09/2021,“PROGRAMA DE TUTORIZACIÓN Y APOYO A LAS EMPRES...,“PROGRAMA DE TUTORIZACIÓN Y APOYO A LAS EMPRES...,None,None,352606,7
784792,583232,NO,CANARIAS,"CONSEJERÍA DE ECONOMÍA, CONOCIMIENTO Y EMPLEO",None,10/09/2021,“PROGRAMA DE TUTORIZACIÓN A LA INTERNACIONALIZ...,“PROGRAMA DE TUTORIZACIÓN A LA INTERNACIONALIZ...,None,None,352606,8
530810,329262,NO,AZUQUECA DE HENARES,AYUNTAMIENTO DE AZUQUECA DE HENARES,None,16/01/2017,CONVENIO CON FUNDACIÓN SOLIDARIDAD DEL HENARES...,zuqueca.sedelectronica.es/transparency/233c004...,None,None,352606,9


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352606 entries, 531256 to 750195
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   1       352606 non-null  object
 1   2       352606 non-null  object
 2   3       352606 non-null  object
 3   4       352606 non-null  object
 4   5       352606 non-null  object
 5   6       352606 non-null  object
 6   7       352606 non-null  object
 7   8       352606 non-null  object
 8   9       352606 non-null  object
 9   10      352606 non-null  object
 10  11      352606 non-null  int64 
 11  12      352606 non-null  int64 
dtypes: int64(2), object(10)
memory usage: 35.0+ MB


NOTA: Para evitar que se nos cree una columna adicional en el dataframe, le hemos indicado en la importación index_col=0 para que utilice como índice el IDConv. Si no lo hacemos así, al generar luego nuestras expectativas, tendremos que tener en cuenta que hay una columna 0, que es índice, que simplemente es un entero que va autoincrementándose.

La salida es más o menos la misma, aunque encontramos más adelante URLs que si están bien formadas, así que habría que revisar los datos originales. En cualquier caso, volvemos a nuestra misión original. Vamos a partir el dataframe en dos ficheros independientes.

In [9]:
df_entrenamiento = df.loc[0:len(df.index)/2-1]
df_verificacion = df.loc[len(df.index)/2:len(df.index)]
print(f'df_entrenamiento contiene {len(df_entrenamiento.index)} elementos. df_pruebas {len((df_verificacion.index))}. En total son {len(df_entrenamiento.index) + len(df_verificacion.index)}.\n')
print(f'El dataframe original contiene {len(df.index)} elementos')

df_entrenamiento contiene 176303 elementos. df_pruebas 176303. En total son 352606.

El dataframe original contiene 352606 elementos


Guardamos los datos parciales en dos csv independientes llamados entrenamiento.csv y verificacion.csv

In [10]:
df_entrenamiento.to_csv('entrenamiento.csv')
df_verificacion.to_csv('verificacion.csv')

# Entrenamiento inicial Great Expectations

El primer paso para trabajar con Great Expectations es crear un Data Context. Para ello ejecutaremos desde el directorio donde tenemos nuestros ficheros: 

```great_expectations init```

Esta es la salida del comando:
```
PS C:\Users\Mi PC\dev\contrib\ge_subvenciones> great_expectations init
Using v3 (Batch Request) API

  ___              _     ___                  _        _   _
 / __|_ _ ___ __ _| |_  | __|_ ___ __  ___ __| |_ __ _| |_(_)___ _ _  ___
| (_ | '_/ -_) _` |  _| | _|\ \ / '_ \/ -_) _|  _/ _` |  _| / _ \ ' \(_-<
 \___|_| \___\__,_|\__| |___/_\_\ .__/\___\__|\__\__,_|\__|_\___/_||_/__/
                                |_|
             ~ Always know what to expect from your data ~

Let's create a new Data Context to hold your project configuration.

Great Expectations will create a new directory with the following structure:

    great_expectations
    |-- great_expectations.yml
    |-- expectations
    |-- checkpoints
    |-- plugins
    |-- .gitignore
    |-- uncommitted
        |-- config_variables.yml
        |-- data_docs
        |-- validations

OK to proceed? [Y/n]:

================================================================================

Congratulations! You are now ready to customize your Great Expectations configuration.

You can customize your configuration in many ways. Here are some examples:

  Use the CLI to:
    - Run `great_expectations datasource new` to connect to your data.
    - Run `great_expectations checkpoint new <checkpoint_name>` to bundle data with Expectation Suite(s) in a Checkpoint for later re-validation.
    - Run `great_expectations suite --help` to create, edit, list, profile Expectation Suites.
    - Run `great_expectations docs --help` to build and manage Data Docs sites.

  Edit your configuration in great_expectations.yml to:
    - Move Stores to the cloud
    - Add Slack notifications, PagerDuty alerts, etc.
    - Customize your Data Docs

Please see our documentation for more configuration options!
```

Una vez que tenemos inicializado el proyecto, es hora de añadir nuestro origen de datos. Para ello ejecutamos:

```
great_expectations datasource new
```

Seleccionamos las siguientes opciones:

```
PS C:\Users\Mi PC\dev\contrib\ge_subvenciones> great_expectations datasource new
Using v3 (Batch Request) API

What data would you like Great Expectations to connect to?
    1. Files on a filesystem (for processing with Pandas or Spark)
    2. Relational database (SQL)
: 1

What are you processing your files with?
1. Pandas
2. PySpark
: 1

Enter the path of the root directory where the data files are stored. If files are on local disk enter a path relative to your current working directory or an absolute path.
: C:\Users\Mi PC\dev\contrib\ge_subvenciones
Please install the optional dependency 'black' to enable linting. Returning input with no changes.
Because you requested to create a new Datasource, we'll open a notebook for you now to complete it!
```

A continuación se nos abre un nuevo cuaderno de Jupyter Notebook para acabar de configurar nuestro origen de datos. 

Configuramos los siguientes parámetros:
```
datasource_name = "subvenciones"
```

Y ejecutamos todas las celdas, para guardar este nuevo datasource. En la salida, observaremos que aparecen listados todos los ficheros csv que tenemos disponibles en el directorio.

```
PS C:\Users\Mi PC\dev\contrib\ge_subvenciones> great_expectations suite new
Using v3 (Batch Request) API

How would you like to create your Expectation Suite?
    1. Manually, without interacting with a sample batch of data (default)
    2. Interactively, with a sample batch of data
    3. Automatically, using a profiler
: 3

A batch of data is required to edit the suite - let's help you to specify it.


Which data asset (accessible by data connector "default_inferred_data_connector_name") would you like to use?
    1. convocatorias2.csv.gz
    2. entrenamiento.csv
    3. ficheros_verificacion
    4. ge_subvenciones.ipynb
    5. great_expectations

Type [n] to see the next page or [p] for the previous. When you're ready to select an asset, enter the index.
: 2

Name the new Expectation Suite [entrenamiento.csv.warning]: entrenamiento_subvenciones

Great Expectations will create a notebook, containing code cells that select from available columns in your dataset and
generate expectations about them to demonstrate some examples of assertions you can make about your data.

When you run this notebook, Great Expectations will store these expectations in a new Expectation Suite "entrenamiento_subvenciones" here:

  file://C:\Users\Mi PC\dev\contrib\ge_subvenciones\great_expectations\expectations/entrenamiento_subvenciones.json

Would you like to proceed? [Y/n]: Y
```

Una vez ejecutemos este comando, se nos abrirá un nuevo Jupyter notebook, que se encargará de lanzar el Auto Profiler para generar las expectativas a partir de los datos de entrenamiento. Una vez acabe de calcular las métricas, abrirá los DataDocs con las expectativas que ha generado, para que podamos revisarlas.

En la primera ejecutacion se nos han generado unas expectativas a nivel de tabla bastante inútiles.
```
Expectation
Must have these columns in this order: Unnamed: 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12
['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
Must have greater than or equal to 176303 and less than or equal to 176303 rows.
```

Volvemos a lanzar el perfilador, para revisar la configuración a fondo con el siguiente comando:

```
great_expectations suite new --profile
```

Llamamos al nuevo perfil entrenamiento_profiler y modificamos el parametro `ignore_columns`para dejarlo vacio y que analice así todas las columnas del documento. Hay columnas que están vacías, pero como no tenemos control sobre el dataset, también vendría bien que nos avisara si derrepente hay campos que estaban vacios y se empiezan a poblar.
 

En total nos ha generado 72 expectativas. Vamos a editarlas con el siguiente comando:

```
great_expectations suite edit entrenamiento_profiler
```

# ¿Cuales son nuestras expectativas respecto a los datos?
Antes de meternos de lleno en el código, vamos a describir a alto nivel nuestras expectativas respecto al código. Comenzaremos haciéndolo de forma burda, para refinarlo en una segunda pasada.

## Indice autogenerado #0
La primera columna es un índice autogenerado al importar el fichero desde pandas. Podemos pasar parámetros al importador, para que no nos agregue esta columna, pero no está del todo claro cómo hacerlo en la nueva documentación de la v3. Por este motivo, vamos a tratarlo como una columna más que cumple las siguientes expectativas.

### Expectativas
Los valores de esta columna son enteros
No hay valores nulos
Los valores van incrementándose en cada registro

## IDConv #1
Esta columna representa el ID de la Convocatoria de Subvención.

### Expectativas
Los valores de esta columna no son nulos - expect_column_values_to_not_be_null
Los valores de esta columa son únicos - expect_column_proportion_of_unique_values_to_be_between min 1.0 max 1.0
Los valores de esta columna son enteros - expect_column_values_to_be_in_type_list 

"expectation_type": "expect_column_values_to_be_in_type_list",
        "meta": {},
        "kwargs": {
            "column": "0",
            "type_list": [
                "INTEGER",
                "integer",
                "int",
                "int_",
                "int8",
                "int16",
                "int32",
                "int64",
                "uint8",
                "uint16",
                "uint32",
                "uint64",
                "Int8Dtype",
                "Int16Dtype",
                "Int32Dtype",
                "Int64Dtype",
                "UInt8Dtype",
                "UInt16Dtype",
                "UInt32Dtype",
                "UInt64Dtype",
                "INT",
                "INTEGER",
                "INT64",
                "TINYINT",
                "BYTEINT",
                "SMALLINT",
                "BIGINT",
                "IntegerType",
                "LongType",

## id #2
Esta columna representa el ID interno de la subvención en el sistema del convocante

### Expectativas
Este campo no debe ser nulo
Este campo es de tipo string


## mrr #3
Esta columna representa si la convocatoria está financiada desde los fondos del Mecanismo de Recuperación y Resiliencia

### Expectativas
Este campo es de tipo boolean y sólo puede contener los valores SI o NO. - expect_column_values_to_be_in_set <- "value_set": ["NO", "SI"]
Este campo no es nulo - expect_column_values_to_not_be_null

## convocanteN1 #4
## convocanteN2 #5
## convocanteN3 #6
Estos tres campos representan los distintos niveles de los distintos convocantes

### Expectativas
Este campo no debe ser nulo
Este campo no debe contener valores únicos
Los valores aceptados deberían estar incluidos dentro de una lista predefinida

## fechareg #7

### Expectativas
Este campo no debe ser nulo
Este campo es de tipo fecha
El formato de esta fecha es dd/mm/yyyy
El año es mayor que 2010 y menor que la fecha actual

## titulo #8
### Expectativas
Este campo no debe ser nulo (aunque existen registros con titulo vacio " ", "  ","...", conformado por varios espacios.)
Este campo es de tipo string


## bbreguladoras #9 - nombre tomado de los ficheros de jurídicas

### Expectativas
Este campo es de tipo URL o string
Puede contener valores repetidos
Puede ser nulo ()

## tituloleng #10
### Expectativas
Este campo puede ser nulo
Este campo es de tipo string

## verConcesiones #11 
Esta columna está compuesta por valores nulos.

### Expectativa

## dummy1 #12
Esta columna recoge el número total de registros devueltos en la búsqueda. En los ficheros originales era 350078. En el fichero más reciente, el número ha incrementado hasta 352606.

### Expectativa

## dummy2 #13
Esta columna recoge el mismo valor de IDConv + 1

### Expectativas 


